In [110]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%matplotlib qt

In [111]:
test_images = ['00001.01417766',
               '00001.03812223',
               '00001.02500618',
               '00001.04331330',
               '00001.06923671',
               '00001.01410167',
               '00002.00337768']

In [434]:
image = cv2.imread('phase1/images/'+ test_images[5] +'.png')
image = cv2.imread('test_data_tehran/34.png')
image = cv2.resize(image, (300,300))

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# gray = cv2.GaussianBlur(gray, (7, 7), 0)


In [435]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
th,image_binary = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
plt.imshow(image_binary)

# road mask

In [436]:
from road_segment import UnetModel

road_seg = UnetModel('unet_epoch_38.pt')
mask = road_seg.detect_mask(image)
new_mask = road_seg.mask2convex(mask)

In [437]:
plt.figure()
plt.subplot(2,2,1)
plt.imshow(mask)
plt.subplot(2,2,2)
plt.imshow(image)
plt.subplot(2,2,3)
plt.imshow(new_mask)
plt.subplot(2,2,4)
plt.imshow(np.expand_dims(new_mask, -1)*image)
plt.show()

In [438]:
new_mask.shape

(300, 300)

In [439]:

np.unique(new_mask)

array([0, 1], dtype=uint8)

### gaussian -> sharp -> canny 

In [440]:
res = cv2.GaussianBlur(image, (3, 3), 0)
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
res = cv2.filter2D(res, -1, kernel)



plt.imshow(res)

In [441]:
canny_image = cv2.Canny(res,300,500)
plt.imshow(canny_image)

In [442]:
res = cv2.bitwise_and(canny_image,canny_image,mask = new_mask)
plt.imshow(res)


### hough

In [476]:
from skimage.transform import probabilistic_hough_line
lines = probabilistic_hough_line(res,threshold=25,line_length=max(res.shape)//24,line_gap=0,seed=42)
# lines = probabilistic_hough_line(res, threshold=25, line_length=15,line_gap=0)
lines = np.array(lines)
lines.shape

(6, 2, 2)

In [477]:
res.shape

(300, 300)

In [478]:
lines = lines.reshape(-1,2,2)
slope = (lines[:,1,1]-lines[:,0,1]) / (lines[:,1,0]-lines[:,0,0])

lines = lines[abs(slope)>0.2]
slope = slope[abs(slope)>0.2]

lines = lines[abs(slope) != np.inf]
slope = slope[abs(slope) != np.inf]
# slope[slope == np.inf] = 10000
# slope[slope == -np.inf] = -10000

In [479]:
y1 = image.shape[0]
x1 = (y1 - lines[:,1,1])/slope + lines[:,1,0]

data = np.vstack([slope,x1]).T

In [480]:
plt.scatter(data[:,0],data[:,1])

# MeanShift

In [481]:
from sklearn.cluster import MeanShift, estimate_bandwidth
bandwidth = estimate_bandwidth(data, quantile=0.2)
bandwidth = bandwidth if bandwidth > 0 else 0.01
clustering = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(data)
clustering.labels_

D:\Users\amirh\anaconda3\lib\site-packages\sklearn\cluster\_mean_shift.py:234: UserWarning: Binning data failed with provided bin_size=0.010000, using data points as seeds.
  warnings.warn("Binning data failed with provided bin_size=%f,"


array([2, 4, 5, 3, 0, 1], dtype=int64)

In [482]:
np.unique(clustering.labels_)

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [483]:
clustering.cluster_centers_

array([[   1.27272727,  482.57142857],
       [  -0.78571429, -235.45454545],
       [  -1.5       ,  -23.33333333],
       [  -1.6       ,  -15.25      ],
       [  -1.63636364,   -7.16666667],
       [  -1.64705882,   -6.39285714]])

In [484]:
final_lines = []
for slope, x in clustering.cluster_centers_:
    if abs(slope)>0.5:
        point1 = [int(x),int(y1)]
        point2 = [int(-y1/slope + x),0]
        final_lines.append([point1,point2])

In [485]:
final_lines = np.array(final_lines)
final_lines.shape

(6, 2, 2)

# plot

In [488]:
start_point = (154, 0)
end_point = (386, 288)
  
color = (255, 255, 0)
  
thickness = 5

new_image = image.copy()
last_mask = np.zeros_like(new_image)

for l in final_lines:
    start_point = (l[0,0], l[0,1])
    end_point = (l[1,0], l[1,1])
    new_image = cv2.line(new_image, start_point, end_point, color, thickness)
    last_mask = cv2.line(last_mask, start_point, end_point, (256,256,256), thickness)
  

plt.figure(figsize=(15,15))
plt.subplot(1,4,1)
plt.imshow(image)
plt.subplot(1,4,2)
plt.imshow(new_image)
plt.subplot(1,4,3)
f_image = cv2.bitwise_and(new_image,new_image,mask = new_mask)
plt.imshow(f_image)
plt.subplot(1,4,4)
# f_image = cv2.bitwise_and(new_image,new_image,mask = new_mask)
plt.imshow(last_mask)

plt.show()

# loop

In [153]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import torch
from road_segment import UnetModel
from sklearn.cluster import MeanShift, estimate_bandwidth
from skimage.transform import probabilistic_hough_line


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
road_seg = UnetModel('unet_epoch_38.pt')

image_addresses = glob('test_data_tehran/*.png')
for img_path in image_addresses:
    try:
        image = cv2.imread(img_path)
        mask = road_seg.detect_mask(image)
        new_mask = road_seg.mask2convex(mask)

        res = cv2.GaussianBlur(image, (3, 3), 0)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        res = cv2.filter2D(res, -1, kernel)

        canny_image = cv2.Canny(res,300,500)
        res = cv2.bitwise_and(canny_image,canny_image,mask = new_mask)

        lines = probabilistic_hough_line(res,threshold=15,line_length=max(res.shape)//24,line_gap=0,seed=42)
        lines = np.array(lines)

        lines = lines.reshape(-1,2,2)
        slope = (lines[:,1,1]-lines[:,0,1]) / (lines[:,1,0]-lines[:,0,0])

        lines = lines[abs(slope)>0.2]
        slope = slope[abs(slope)>0.2]

        lines = lines[abs(slope) != np.inf]
        slope = slope[abs(slope) != np.inf]

        y1 = image.shape[0]
        x1 = (y1 - lines[:,1,1])//slope + lines[:,1,0]

        data = np.vstack([slope,x1]).T
        bandwidth = estimate_bandwidth(data, quantile=0.2)
        bandwidth = bandwidth if bandwidth > 0 else 0.01
        clustering = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(data)

        final_lines = []
        for slope, x in clustering.cluster_centers_:
            if abs(slope)>0.5:
                point1 = [int(x),int(y1)]
                point2 = [int(-y1/slope + x),0]
                final_lines.append([point1,point2])

        final_lines = np.array(final_lines)
        
        color = (0, 255, 255)
        thickness = 3
        new_image = image.copy()

        for l in final_lines:
            start_point = (l[0,0], l[0,1])
            end_point = (l[1,0], l[1,1])
            new_image = cv2.line(new_image, start_point, end_point, color, thickness)

        cv2.imwrite(img_path.replace('.png','_res.png'),new_image)
    except:
        pass


D:\Users\amirh\anaconda3\lib\site-packages\sklearn\cluster\_mean_shift.py:234: UserWarning: Binning data failed with provided bin_size=0.010000, using data points as seeds.
  warnings.warn("Binning data failed with provided bin_size=%f,"
